In [11]:
import os
import librosa
import soundfile as sf
from librosa.core import load, stft, istft, magphase
from config import *
from concurrent.futures import ThreadPoolExecutor
from time import time
import asyncio
import numpy as np
from multiprocessing import cpu_count
from pydub import AudioSegment

In [26]:
dirs = list(os.walk('MedleyDB_V2/'))[0][1]

def downsample(input_path, output_path):
    wav, _ = librosa.load(input_path, sr=SAMPLE_RATE)
    sf.write(output_path, wav, SAMPLE_RATE)
#     librosa.output.write_wav(output_path, wav, SAMPLE_RATE, norm=True)
    print(f"Saving {output_path}")


In [31]:
if __name__ == '__main__':
    with ThreadPoolExecutor(max_workers=cpu_count() * 2) as pool:
        for i in range(len(dirs)):
            files = os.listdir(f'MedleyDB_V2/{dirs[i]}/')
            target_dir = 'MedleyDB_V2_resized/{}/'.format(dirs[i])
            # check if there are .mp3 files and change to .wav
            src = str("MedleyDB_V2/"+dirs[i]+"/"+files[1])
            dst = str("MedleyDB_V2/"+dirs[i]+"/"+dirs[i] + "_MIX.wav")
            if(".mp3" in files [1]):
                sound = AudioSegment.from_mp3(src)
                sound.export(dst, format="wav")
                
            os.makedirs(target_dir, exist_ok=True)
            if len(files) == 2:
                pool.submit(downsample, f'MedleyDB_V2/{dirs[i]}/{files[0]}', target_dir + files[0])
                pool.submit(downsample, f'MedleyDB_V2/{dirs[i]}/{files[1]}', target_dir + files[1])
            elif len(files) == 3:
                pool.submit(downsample, f'MedleyDB_V2/{dirs[i]}/{files[0]}', target_dir + files[0])
                pool.submit(downsample, f'MedleyDB_V2/{dirs[i]}/{files[1]}', target_dir + files[1])
                pool.submit(downsample, f'MedleyDB_V2/{dirs[i]}/{files[2]}', target_dir + files[2])
            else:
                pool.submit(downsample, f'MedleyDB_V2/{dirs[i]}/{files[0]}', target_dir + files[0])
                pool.submit(downsample, f'MedleyDB_V2/{dirs[i]}/{files[2]}', target_dir + files[2])
                pool.submit(downsample, f'MedleyDB_V2/{dirs[i]}/{files[3]}', target_dir + files[3])

Saving MedleyDB_V2_resized/BarefootSisters_RedJetta/BarefootSisters_RedJetta_Accompaniment.wav
Saving MedleyDB_V2_resized/BarefootSisters_RedJetta/BarefootSisters_RedJetta_MIX.wav
Saving MedleyDB_V2_resized/BarefootSisters_RedJetta/BarefootSisters_RedJetta_Vocal.wav
Saving MedleyDB_V2_resized/CassandraJenkins_PerfectDay/CassandraJenkins_PerfectDay_Accompaniment.wav
Saving MedleyDB_V2_resized/CassandraJenkins_PerfectDay/CassandraJenkins_PerfectDay_MIX.wav
Saving MedleyDB_V2_resized/CassandraJenkins_PerfectDay/CassandraJenkins_PerfectDay_Vocal.wav
Saving MedleyDB_V2_resized/DahkaBand_SoldierMan/DahkaBand_SoldierMan_Accompaniment.wav
Saving MedleyDB_V2_resized/DahkaBand_SoldierMan/DahkaBand_SoldierMan_MIX.wav
Saving MedleyDB_V2_resized/DahkaBand_SoldierMan/DahkaBand_SoldierMan_Vocal.wav
Saving MedleyDB_V2_resized/DeadMilkmen_PrisonersCinema/DeadMilkmen_PrisonersCinema_Accompaniment.wav
Saving MedleyDB_V2_resized/DeadMilkmen_PrisonersCinema/DeadMilkmen_PrisonersCinema_MIX.wav
Saving Medley

KeyboardInterrupt: 

In [23]:
def load_as_mag(file):
    wav, _ = librosa.load(file, sr=None)
    spectrogram = stft(wav, n_fft=WINDOW_SIZE, hop_length=HOP_LENGTH)
    mag, _ = magphase(spectrogram)
    return mag.astype(np.float32)

def save_to_npz(base, sample):
    nps = {}
    file = os.listdir(f'{base}/{sample}/')
    if len(file) == 2:
        acc = load_as_mag(f'{base}/{sample}/{file[0]}')
        mix = load_as_mag(f'{base}/{sample}/{file[1]}')
    else:
        acc = load_as_mag(f'{base}/{sample}/{file[0]}')
        mix = load_as_mag(f'{base}/{sample}/{file[1]}')
        vocal = load_as_mag(f'{base}/{sample}/{file[2]}')
        
    mix_max = mix.max()
    mix_norm = mix / mix_max
    vocal_norm = vocal / mix_max
    acc_norm = acc / mix_max
    
    print(f"Saving {sample}")
    if len(file) == 2:
        np.savez_compressed(f'MedleyDB_Npz/{sample}.npz', mix=mix_norm, acc=acc_norm)
    else:
        np.savez_compressed(f'MedleyDB_Npz/{sample}.npz', mix=mix_norm, vocal=vocal_norm, acc=acc_norm)
        
if __name__ == '__main__':
    if not os.path.exists('MedleyDB_Npz'):
        os.makedirs('MedleyDB_Npz')
    dirs = sorted(list(os.walk('MedleyDB_V2_resized'))[0][1])
    with ThreadPoolExecutor(max_workers=cpu_count() * 2) as pool:
        for i in range(len(dirs)):
            pool.submit(save_to_npz, 'MedleyDB_V2_resized', dirs[i])

Saving TleilaxEnsemble_Late
